In [11]:
# ==============================================================
# BLOCO 1 — Consulta Athena com retorno via boto3
# --------------------------------------------------------------
# Esse bloco complementa o teste anterior (Bloco 1),
# mostrando como capturar e exibir o resultado completo
# de uma query Athena executada via boto3.
#
# A diferença é que aqui usamos o método `get_query_results`
# com o QueryExecutionId retornado anteriormente,
# permitindo recuperar as linhas e colunas processadas
# diretamente do Athena, sem precisar acessar o console.
#
# Essa etapa demonstra a execução completa:
#   boto3.start_query_execution() → boto3.get_query_results()
# ==============================================================

import boto3
import pandas as pd

# Cria cliente do serviço Athena
athena = boto3.client("athena")

# Define e executa a query
query = """
SELECT * FROM "b3_refined_db"."refined"
WHERE year = '2025'
ORDER BY day DESC
LIMIT 10;
"""

response = athena.start_query_execution(
    QueryString=query,
    QueryExecutionContext={'Database': 'b3_refined_db'},
    ResultConfiguration={'OutputLocation': 's3://b3-dataflow-logs/athena-results/'}
)

# Captura o ID da execução
query_execution_id = response["QueryExecutionId"]
print(f"🟢 Query enviada com sucesso! ID: {query_execution_id}")

# Aguarda a execução concluir antes de buscar os resultados
import time
state = "RUNNING"
while state in ["RUNNING", "QUEUED"]:
    result = athena.get_query_execution(QueryExecutionId=query_execution_id)
    state = result["QueryExecution"]["Status"]["State"]
    if state == "FAILED":
        print("❌ Falha na execução da query.")
        break
    elif state == "SUCCEEDED":
        print("✅ Execução concluída com sucesso.")
        break
    time.sleep(2)

# Recupera os resultados
results = athena.get_query_results(QueryExecutionId=query_execution_id)

# Converte o resultado para DataFrame
columns = [col["Label"] for col in results["ResultSet"]["ResultSetMetadata"]["ColumnInfo"]]
rows = [row["Data"] for row in results["ResultSet"]["Rows"][1:]]  # ignora cabeçalho
data = [[cell.get("VarCharValue", None) for cell in row] for row in rows]
df_preview = pd.DataFrame(data, columns=columns)

# Exibe as 10 primeiras linhas
display(df_preview.head(10))


🟢 Query enviada com sucesso! ID: 7aa5cfd9-f1f7-4874-8379-5f4265959dbe
✅ Execução concluída com sucesso.


,date,preco_fechamento,high,low,preco_abertura,volume_negociado,ticker,media_movel_3d,year,month,day
0,2025-09-11 00:00:00.000,143151.0,144013.0,142349.0,142349.0,7570400,IBOV,None,2025,10,12
1,2025-09-12 00:00:00.000,142272.0,143202.0,142241.0,143151.0,6388600,IBOV,142590.66666666666,2025,10,12
2,2025-09-15 00:00:00.000,143547.0,144194.0,142292.0,142292.0,6614000,IBOV,142990.0,2025,10,12
3,2025-09-16 00:00:00.000,144062.0,144584.0,143547.0,143547.0,8478200,IBOV,143293.66666666666,2025,10,12
4,2025-09-17 00:00:00.000,145594.0,146331.0,143910.0,144059.0,9604400,IBOV,144401.0,2025,10,12
5,2025-09-18 00:00:00.000,145500.0,145726.0,144993.0,145594.0,8372000,IBOV,145052.0,2025,10,12
6,2025-09-19 00:00:00.000,145865.0,146399.0,145496.0,145500.0,10279800,IBOV,145653.0,2025,10,12
7,2025-09-22 00:00:00.000,145109.0,145864.0,144117.0,145864.0,9428100,IBOV,145491.33333333334,2025,10,12
8,2025-09-23 00:00:00.000,146425.0,147179.0,145107.0,145112.0,8143800,IBOV,145799.66666666666,2025,10,12
9,2025-09-10 00:00:00.000,142349.0,143182.0,141612.0,141612.0,7138700,IBOV,None,2025,10,12


In [9]:
# ==============================================================
# BLOCO 2 — Query analítica com cálculo de volatilidade e amplitude
# --------------------------------------------------------------
# Essa célula executa uma consulta no Amazon Athena via awswrangler
# e retorna os dados refinados em formato pandas DataFrame.
#
# O objetivo é enriquecer os dados do índice IBOV com duas novas
# features derivadas:
#   • 'amplitude' — diferença entre o preço máximo e mínimo do dia.
#   • 'volatilidade_pct' — variação percentual relativa ao fechamento.
#
# Essas variáveis servem como base para análises estatísticas e
# modelos de detecção de anomalias, como o Isolation Forest,
# além de outras técnicas de feature engineering (ex: rolling mean,
# z-score, desvio padrão móvel, etc).
#
# Essa etapa consolida o pipeline end-to-end:
#   S3 → Glue → Athena → Python (pandas + awswrangler)
# ==============================================================

import boto3
import pandas as pd
import awswrangler as wr

# Configurações
database = "b3_refined_db"
table = "refined"
output_path = "s3://b3-dataflow-logs/athena-results/"

# Consulta base
query = f"""
SELECT
    date,
    preco_abertura,
    preco_fechamento,
    high,
    low,
    volume_negociado,
    (high - low) AS amplitude,
    ROUND(((high - low) / preco_fechamento) * 100, 2) AS volatilidade_pct
FROM "{database}"."{table}"
WHERE year = '2025';
"""

# Executa query e lê diretamente em pandas
df = wr.athena.read_sql_query(
    sql=query,
    database=database,
    ctas_approach=False
)

# Visualiza as primeiras linhas
display(df.head(10))


,date,preco_abertura,preco_fechamento,high,low,volume_negociado,amplitude,volatilidade_pct
0,2025-09-10,141612.0,142349.0,143182.0,141612.0,7138700,1570.0,1.10
1,2025-09-11,142349.0,143151.0,144013.0,142349.0,7570400,1664.0,1.16
2,2025-09-12,143151.0,142272.0,143202.0,142241.0,6388600,961.0,0.68
3,2025-09-15,142292.0,143547.0,144194.0,142292.0,6614000,1902.0,1.33
4,2025-09-16,143547.0,144062.0,144584.0,143547.0,8478200,1037.0,0.72
5,2025-09-17,144059.0,145594.0,146331.0,143910.0,9604400,2421.0,1.66
6,2025-09-18,145594.0,145500.0,145726.0,144993.0,8372000,733.0,0.50
7,2025-09-19,145500.0,145865.0,146399.0,145496.0,10279800,903.0,0.62
8,2025-09-22,145864.0,145109.0,145864.0,144117.0,9428100,1747.0,1.20
9,2025-09-23,145112.0,146425.0,147179.0,145107.0,8143800,2072.0,1.42


In [10]:
# ================================================================
# BLOCO 3 – Query agregada de estatísticas do dataset refinado
# ---------------------------------------------------------------
# Essa célula executa uma consulta no Amazon Athena via awswrangler,
# retornando estatísticas descritivas consolidadas sobre o índice IBOV.
#
# O objetivo é calcular medidas agregadas (média, máximo e mínimo)
# a partir dos dados refinados, servindo para validar a consistência
# dos valores e a integridade das transformações feitas no Glue Job.
#
# Essa consulta garante que as features derivadas no dataset refinado
# (como amplitude e volatilidade_pct) estão coerentes com a escala
# e o comportamento esperado do índice IBOV.
#
# Em resumo, essa query é uma checagem de qualidade dos dados,
# não uma análise preditiva.
# ================================================================

import awswrangler as wr

query = """
SELECT 
    AVG(preco_fechamento) AS media_fechamento,
    MAX(high) AS max_high,
    MIN(low) AS min_low
FROM "b3_refined_db"."refined"
WHERE year = '2025';
"""

# Executa query agregada e retorna resultado em DataFrame
df_stats = wr.athena.read_sql_query(
    sql=query,
    database="b3_refined_db",
    ctas_approach=False
)

# Exibe o resultado agregado
display(df_stats)


,media_fechamento,max_high,min_low
0,144211.217391,147578.0,140231.0


In [12]:
# ==============================================================
# BLOCO 4 — Feature Engineering / Preparação para Isolation Forest
# --------------------------------------------------------------
# Essa célula realiza o pré-processamento dos dados retornados
# pelo Bloco 2, preparando o DataFrame para aplicações de
# detecção de anomalias (ex: Isolation Forest).
#
# O foco é gerar novas features derivadas de volatilidade e amplitude,
# capturar tendências de curto prazo e normalizar variáveis
# para posterior treinamento de modelos de machine learning.
#
# As principais transformações incluem:
#   • Cálculo da média móvel de 3 dias da volatilidade (rolling_mean)
#   • Cálculo do desvio padrão móvel (rolling_std)
#   • Cálculo do z-score de volatilidade (padronização)
#   • Criação de coluna binária indicando picos de volatilidade
#
# Essa etapa consolida o pipeline analítico, tornando o dataset
# apto para modelagem estatística e machine learning supervisionado
# ou não supervisionado (ex: Isolation Forest, DBSCAN, PCA).
# ==============================================================

import pandas as pd
import numpy as np

# Cópia do DataFrame base (retornado do Bloco 2)
df_features = df.copy()

# Garante que as colunas numéricas estejam no formato float
numeric_cols = ["preco_abertura", "preco_fechamento", "high", "low", "volume_negociado", "amplitude", "volatilidade_pct"]
df_features[numeric_cols] = df_features[numeric_cols].astype(float)

# Calcula médias e desvios móveis (janela de 3 dias)
df_features["vol_mean_3d"] = df_features["volatilidade_pct"].rolling(window=3).mean()
df_features["vol_std_3d"] = df_features["volatilidade_pct"].rolling(window=3).std()

# Calcula o z-score da volatilidade (padronização)
df_features["vol_zscore"] = (
    (df_features["volatilidade_pct"] - df_features["volatilidade_pct"].mean()) /
    df_features["volatilidade_pct"].std()
)

# Identifica picos de volatilidade (ex: 1.5 desvios acima da média)
df_features["volatilidade_pico"] = np.where(df_features["vol_zscore"] > 1.5, 1, 0)

# Exibe o preview final
display(df_features.head(10))

# Mostra as novas colunas criadas
print("\n🧩 Colunas derivadas criadas:")
print(["vol_mean_3d", "vol_std_3d", "vol_zscore", "volatilidade_pico"])


,date,preco_abertura,preco_fechamento,high,low,volume_negociado,amplitude,volatilidade_pct,vol_mean_3d,vol_std_3d,vol_zscore,volatilidade_pico
0,2025-09-10,141612.0,142349.0,143182.0,141612.0,7138700.0,1570.0,1.10,NaN,NaN,0.129145,0
1,2025-09-11,142349.0,143151.0,144013.0,142349.0,7570400.0,1664.0,1.16,NaN,NaN,0.281469,0
2,2025-09-12,143151.0,142272.0,143202.0,142241.0,6388600.0,961.0,0.68,0.980000,0.261534,-0.937128,0
3,2025-09-15,142292.0,143547.0,144194.0,142292.0,6614000.0,1902.0,1.33,1.056667,0.337095,0.713056,0
4,2025-09-16,143547.0,144062.0,144584.0,143547.0,8478200.0,1037.0,0.72,0.910000,0.364280,-0.835578,0
5,2025-09-17,144059.0,145594.0,146331.0,143910.0,9604400.0,2421.0,1.66,1.236667,0.476900,1.550841,1
6,2025-09-18,145594.0,145500.0,145726.0,144993.0,8372000.0,733.0,0.50,0.960000,0.616117,-1.394101,0
7,2025-09-19,145500.0,145865.0,146399.0,145496.0,10279800.0,903.0,0.62,0.926667,0.637913,-1.089452,0
8,2025-09-22,145864.0,145109.0,145864.0,144117.0,9428100.0,1747.0,1.20,0.773333,0.374344,0.383019,0
9,2025-09-23,145112.0,146425.0,147179.0,145107.0,8143800.0,2072.0,1.42,1.080000,0.413280,0.941543,0



🧩 Colunas derivadas criadas:
['vol_mean_3d', 'vol_std_3d', 'vol_zscore', 'volatilidade_pico']
